# LangChain: Agents

## Outline:

* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools

In [8]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

llm_model = "gpt-4o-mini"

llm = ChatOpenAI(temperature=0.0, model=llm_model)


## Built-in LangChain tools

In [42]:
%pip install wikipedia
%pip install numexpr

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [48]:
from langchain.agents import create_agent
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_classic.chains import LLMMathChain
from langchain_core.tools import Tool

llm = ChatOpenAI(temperature=0.0, model=llm_model)

# Create the LLMMathChain
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

# Define the tool for your agent
math = Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="Useful for when you need to answer questions about math.",
    )

# Step 2: Set up the Wikipedia tool
wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# Step 3: Create the agent
agent = create_agent(
    model=llm,
    tools=[wikipedia_tool, math],
    debug=True
)

# Test math question
agent.invoke({
        "messages": [{"role": "user", "content": "What is 25% of 550? Output only the mathematical value, not the reasoning to compute the value."}]
    })

# Test wiki question
agent.invoke({
        "messages": [{"role": "user", "content": "Who is the inventor of modern physics?"}]
    })


[values] {'messages': [HumanMessage(content='What is 25% of 550? Output only the mathematical value, not the reasoning to compute the value.', additional_kwargs={}, response_metadata={}, id='868ca140-bef1-41cd-b3f7-dece7a72b37e')]}
[updates] {'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 135, 'total_tokens': 155, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CaT12t18Uvr517MAUx0kewK3Yhv4K', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--ce93059b-aeec-45ff-9f8d-76fb6b022416-0', tool_calls=[{'name': 'Calculator', 'args': {'__arg1': '25% of 550'}, 'id': 'call_

{'messages': [HumanMessage(content='Who is the inventor of modern physics?', additional_kwargs={}, response_metadata={}, id='40f4d1a1-69f7-43e6-94d9-1797412e7a38'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 120, 'total_tokens': 137, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CaT15odADZ78fkenfEkea0Um6MPhN', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--e136fbc5-1971-4da7-81cb-53e0173c6f20-0', tool_calls=[{'name': 'wikipedia', 'args': {'query': 'inventor of modern physics'}, 'id': 'call_itSSNgxgnkcNrr59EDPmMfPs', 'type': 'tool_call'}], usage_metadata={'input_tokens': 120

## Python Agent

In [57]:
from langchain_experimental.tools import PythonREPLTool
from langchain_experimental.agents.agent_toolkits import create_python_agent

agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [58]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [60]:
agent.run(f"""Sort these customers by \
last name \
and print the output: {customer_list}""") 



> Entering new AgentExecutor chain...
To sort the customers by their last names, I will extract the last names from each sublist and then sort the entire list based on those last names. 

Action: Python_REPL  
Action Input: 
```python
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted_customers = sorted(customers, key=lambda x: x[1])
print(sorted_customers)
```  
Observation: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

Thought:I now know the final answer
Final Answer: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

> Finished chain.


"[['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]"

#### View detailed outputs of the chains

In [61]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False



> Entering new AgentExecutor chain...
To sort the customers by last name and then by first name, I will use Python's built-in sorting capabilities. I will sort the list of customers using a custom key that sorts first by the last name and then by the first name. 

Action: Python_REPL  
Action Input: `customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]; sorted_customers = sorted(customers, key=lambda x: (x[0], x[1])); print(sorted_customers)`  
Observation: [['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Jen', 'Ayai'], ['Lang', 'Chain'], ['Trance', 'Former']]

Thought:I now know the final answer
Final Answer: [['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Jen', 'Ayai'], ['Lang', 'Chain'], ['Trance', 'Former']]

> Finished chain.


## Define your own tool

In [62]:
#!pip install DateTime

In [65]:
from langchain_classic.agents import tool
from datetime import date

In [72]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [85]:
agent= create_agent(
    tools=[time], 
    model=llm, 
    debug=False)

try:
    result = agent.invoke({
        "messages": [{"role": "user", "content": "whats the date today?"}]
    })

except Exception as e: 
    print(e)

result

{'messages': [HumanMessage(content='whats the date today?', additional_kwargs={}, response_metadata={}, id='be920f83-3816-455a-aa31-707d6fd23582'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 90, 'total_tokens': 103, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CaTEeP5Ec36QVgVr4UwsguwyBLgPt', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--627664db-284b-41e3-a8ce-e10aad9ba751-0', tool_calls=[{'name': 'time', 'args': {'text': ''}, 'id': 'call_Y98cNehsz74ckFPH8w2TYTtH', 'type': 'tool_call'}], usage_metadata={'input_tokens': 90, 'output_tokens': 13, 'total_tokens': 103, 'input_